### Features

Based on the method developed by Weller et al. 2006 (Modelling the spatial distribution of volcanoes: an example form Armenia. In: IAVCEI Spec. Pub. 1, Statistics in volcanology)

1. Calculates probability of vent opening at user specified points during an eruption assuming a decaying probability away from previously active vents.
2. Rate of probability decay is assumed to be Gaussian. The bandwidth of this Guassian kernel is estimated using the user defined vent location data. Nearest neighbour distances of vents are calucated and their cumulative density distribution of distance is compared to expected Gaussian distributions. The bandwidth of the Gaussian kernel is optimised to produce the best fit between the expected gaussian distribution and the user defined data.
3. Probability at each particular location is calculated as the sum of probabilities associated with proximity to every vent, with a fixed bandwidth defined by step 2.

### Import Libraries

In [ ]:
import matplotlib.pyplot as plt
import csv
import numpy as np
from scipy.special import erf
import matplotlib.mlab as mlab
import pandas as pd
from scipy.optimize import curve_fit

### Import Aluto data

In [ ]:
vent = pd.read_csv('/aluto_vents.csv')# import csv of existing vent locations
fishnet = pd.read_csv('/vent_loc.csv') # import csv of input features (points at which to assess probability) 

### Calculate Nearest Neighbour Distances of Vents

In [ ]:
x_mv1 = vent.x_mv.values # set x location values from vent csv
y_mv1 = vent.y_mv.values # set y location values from vent csv

x_mv2 = x_mv1 # create copy of x values
y_mv2 = y_mv1 # create copy of y values

#create empty fields for dx and dy
dx=[] 
dy=[]

#calculate difference in x and y coordinates between every vent. Append these to lists.
for i in range(0,len(vent)):
    for j in range(0,len(vent)):
        dx.append(np.abs(x_mv1[i] - x_mv2[j]))
        dy.append(np.abs(y_mv1[i] - y_mv2[j]))

dist = np.hypot(dx,dy) # calculate distance between vents
by_vent = np.reshape(dist,(len(vent),-1)) # split union of vent distances into vent distances per vent
m_by_vent = np.ma.masked_equal(by_vent, 0.0, copy=False) # mask zero values in array
near = np.amin(m_by_vent, axis=1) # extract nearest (min) distance within each by_vent
nearkm = near*0.001 # convert to km

### Compute the best fit gaussian kernel of the cumulative nearest neighbour distances

In [ ]:
#----------Define function that describes best-fit curve

def func_gauss(kernel,band):
    return erf(kernel/(np.sqrt(2)*band))

#----------Calculate cumulative density distribution of distance to nearest neighbour vent and find best fit-curve

freq = np.histogram(nearkm,bins=1000) # bin near distance data
cum_freq = np.cumsum(freq[0]) # calculate cumulative frequency of near distance bins
cum_frac = cum_freq/max(cum_freq) # calculate cumulative frequency as fraction of total vents
cum_frac = np.insert(cum_frac,0,0) # add zero value to beggining of cum_frac array to account for binning
               
xdata = freq[1] # define x data
ydata = cum_frac # define y data
               
popt, pcov = curve_fit(func_gauss, xdata, ydata) # find curve defined by function that best matches cumulative density distribution

#----------Plot result

plt.plot(xdata,ydata, label='Cumulative density distribution of user data')
plt.plot(xdata, func_gauss(xdata, *popt), 'r-', label="Bandwidth = "+ str(popt)+ " km")
plt.xlabel("Nearest Neighbour Distance (km)")
plt.ylabel("Fraction of Total Vents")
plt.title("Bandwidth = " +str(popt))
plt.show()

### Calculate the vent opening probability based on the bandwidth of the gaussian kernel 

In [ ]:
#------------Extract x and y coordinates from near and input CSVs

x_mf = fishnet.x_mf.values
x_mv = vent.x_mv.values

y_mf = fishnet.y_mf.values
y_mv = vent.y_mv.values

#------------create empty fields for dx and dy
dx=[] 
dy=[]

#------------calculate distance between every input and near feature. Append these to lists.
for i in range(0,len(x_mf)):
    for j in range(0,len(x_mv)):
        dx.append(np.abs(x_mf[i] - x_mv[j]))
        dy.append(np.abs(y_mf[i] - y_mv[j]))

dist = np.hypot(dx,dy) # calculate distance between input and near features
neardist = dist*0.001 # convert to km

#------------calculate vent opening probability at every input feature (fishnet point)

h = popt # set fixed bandwidth as Gaussian best-fit kernel bandwidth estimated in previous step
z = np.exp(-0.5*(neardist/h)**2) # calculate z for every input-near distance
by_fishnet = np.reshape(z,(len(fishnet),-1)) # split list of z values into chunks where each chunk contains all z values associated with each input feature
sumz = np.sum(by_fishnet, axis = 1) # sum z values in each chunk.
intensity = (sumz*(1/(2*np.pi*len(vent)*h**2)))/(4) # calculate spatial intensity for every input feature final division takes into account 500m point spacing but 1 km analysis size ((1km / spacing) ^2)

#------------create dataframe of probabilities at each input location
d = {'xUTM' :x_mf, 'yUTM' :y_mf, 'Prob' :intensity}
df_prob = pd.DataFrame(d)

#------------Visualise the results
plt.scatter(x_mf, y_mf, c=intensity, cmap='viridis', s=7, marker='o')
plt.axis('equal')
plt.colorbar(label='P(vent|eruption)')
plt.xlabel('UTMx [m]')
plt.ylabel('UTMy [m]')

#plt.scatter(x_mv, y_mv, c='white', s=2, marker='^')
plt.ticklabel_format(axis='both', style='sci')#, scilimits=(-2,2))
plt.show()